# Setup

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, scheduler, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}
    
    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, scheduler, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for 5 epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval, )

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

In [2]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102018 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
test_data = EMNISTDataProvider('test', batch_size=batch_size, rng=rng)

KeysView(<numpy.lib.npyio.NpzFile object at 0x110810940>)
KeysView(<numpy.lib.npyio.NpzFile object at 0x1108102b0>)


In [3]:
from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule, GradientDescentLearningRule, RMSPropLearningRule
from mlp.optimisers import Optimiser
from mlp.schedulers import ConstantLearningRateScheduler

# Experiment

In [ ]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

#setup hyperparameters
learning_rate = 1e-5
num_epochs = 100
stats_interval = 1
input_dim, output_dim, hidden_dim = 784, 47, 100
decay_rates = 0.9

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init),
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

scheduler = ConstantLearningRateScheduler(learning_rate)

error = CrossEntropySoftmaxError()
# Use a basic gradient descent learning rule
learning_rule = RMSPropLearningRule(learning_rate=learning_rate, beta=0.9)

#Remember to use notebook=False when you write a script to be run in a terminal
stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
        model, error, learning_rule, train_data, scheduler, test_data, num_epochs, stats_interval, notebook=True)

fig_1.tight_layout()
fig_1_name = "rms_fig_1_test_set.pdf"
fig_1.savefig(fig_1_name)

fig_2.tight_layout()
fig_2_name = "rms_fig_2_test_set.pdf"
fig_2.savefig(fig_2_name)

print('-' * 80)
print('decay_rate={0:.4f}'
      .format(decay_rate))
print('-' * 80)

plt.show()

print('    final error(train) = {0:.2e}'.format(stats[-1, keys['error(train)']]))
print('    final error(valid) = {0:.2e}'.format(stats[-1, keys['error(valid)']]))
print('    final acc(train)   = {0:.2e}'.format(stats[-1, keys['acc(train)']]))
print('    final acc(valid)   = {0:.2e}'.format(stats[-1, keys['acc(valid)']]))
print('    run time per epoch = {0:.2f}'.format(run_time * 1. / num_epochs))

Epoch 1: 4.5s to complete
    error(train)=3.68e+00, acc(train)=9.23e-02, error(valid)=3.69e+00, acc(valid)=9.47e-02


Epoch 2: 4.1s to complete
    error(train)=3.40e+00, acc(train)=1.87e-01, error(valid)=3.41e+00, acc(valid)=1.82e-01


Epoch 3: 4.0s to complete
    error(train)=3.05e+00, acc(train)=2.93e-01, error(valid)=3.06e+00, acc(valid)=2.87e-01


Epoch 4: 3.8s to complete
    error(train)=2.69e+00, acc(train)=3.73e-01, error(valid)=2.71e+00, acc(valid)=3.66e-01


Epoch 5: 3.9s to complete
    error(train)=2.38e+00, acc(train)=4.24e-01, error(valid)=2.40e+00, acc(valid)=4.18e-01


...

Epoch 58: 4.2s to complete
    error(train)=8.97e-01, acc(train)=7.41e-01, error(valid)=9.40e-01, acc(valid)=7.23e-01


Epoch 59: 4.8s to complete
    error(train)=8.91e-01, acc(train)=7.43e-01, error(valid)=9.35e-01, acc(valid)=7.23e-01


Epoch 60: 4.7s to complete
    error(train)=8.86e-01, acc(train)=7.44e-01, error(valid)=9.31e-01, acc(valid)=7.25e-01


Epoch 61: 3.6s to complete
    error(train)=8.80e-01, acc(train)=7.45e-01, error(valid)=9.25e-01, acc(valid)=7.26e-01


Epoch 62: 3.8s to complete
    error(train)=8.75e-01, acc(train)=7.47e-01, error(valid)=9.19e-01, acc(valid)=7.28e-01
